# Calling AlphaFold2 NIM API

This Jupyter Notebook demonstrates how to call the AF2 NIM's predict-structure-from-sequence API to predict a protein structure given an input amino acid sequence, with the AF2 NIM hosted on the same Brev instance where this Jupyter Notebook is running. It primarily follows the [AF2 NIM QuickStart Guide](https://docs.nvidia.com/nim/bionemo/alphafold2/latest/quickstart-guide.html). Please refer to the full [AF2 NIM documentation](https://docs.nvidia.com/nim/bionemo/alphafold2/latest/overview.html) for more details on calling all available AF2 NIM's APIs.

In [1]:
!pip install requests

Defaulting to user installation because normal site-packages is not writeable


In [2]:
AF2_HOST = "http://localhost:8000"

### API Health Check

In [3]:
url = f'{AF2_HOST}/v1/health/ready'
print(url)

http://localhost:8000/v1/health/ready


In [4]:
import requests

url = f'{AF2_HOST}/v1/health/ready'

headers = {
    "content-type": "application/json"
}
try:
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.ok:
        print("Request succeeded:", response.json())
    else:
        print("Request failed:", response.status_code, response.text)
except Exception as E:
    print("Request failed:", E)

Request succeeded: {'status': 'ready'}


### Predict structure - use Jackhmmer for MSA
- For the first API call, I recommend trying the sequence "MNVIDIAIAMAI" and running the code cell below as it is to quickly check if the API is working. You should receive results in less than 30 minutes if you're using the compute configuration described in the [tutorial](https://github.com/hw-ju/Tutorial_AF2_NIM_Brev/blob/main/README.md).
- Please refer to [AF2 NIM Endpoints](https://docs.nvidia.com/nim/bionemo/alphafold2/latest/endpoints.html#) for all available options that can be included in the requests.
- After running the code cell below, run `tail -f af2_nim.log` in your local terminal connected to this Brev instance, and you should start seeing logs like the ones below:
  ```
  "timestamp": "2025-03-07 01:06:24,206", "level": "INFO", "message": "nim_api_post_call_protein_structure_alphafold2_predict_msa_from_sequence_post called"
  "timestamp": "2025-03-07 01:06:24,206", "level": "INFO", "message": "Beginning MSA."
  {"timestamp": "2025-03-07T01:06:24.206908", "level": "INFO", "logger": "foldnim_wrappers", "message": "Beginning query."}
  ...
  ```
  See [af2_nim_uniref90_jackhmmer.log](https://github.com/hw-ju/Tutorial_AF2_NIM_Brev/blob/main/af2_nim_uniref90_jackhmmer.log) for a sample of the complete log generated when running the code cell below as it is.

In [6]:
import requests
import json

url = f'{AF2_HOST}/protein-structure/alphafold2/predict-structure-from-sequence'

# Replace with the actual sequence value.
sequence = "MNVIDIAIAMAI"  

headers = {
    "content-type": "application/json"
}

data = {
    "sequence": sequence,
    "databases": ["uniref90"],
    "algorithm": "jackhmmer"
}

response = requests.post(url, headers=headers, data=json.dumps(data))

# Check if the request was successful
if response.ok:
    print("Request succeeded:", response.json())
else:
    print("Request failed:", response.status_code, response.text)

Request succeeded: ['ATOM      1  N   MET A   1     -14.372   7.964 -15.167  1.00 62.14           N  \nATOM      2  H   MET A   1     -14.143   7.613 -16.086  1.00 62.14           H  \nATOM      3  H2  MET A   1     -13.882   8.834 -15.010  1.00 62.14           H  \nATOM      4  H3  MET A   1     -15.367   8.126 -15.103  1.00 62.14           H  \nATOM      5  CA  MET A   1     -13.960   6.975 -14.155  1.00 62.14           C  \nATOM      6  HA  MET A   1     -14.147   7.350 -13.149  1.00 62.14           H  \nATOM      7  C   MET A   1     -12.470   6.807 -14.323  1.00 62.14           C  \nATOM      8  CB  MET A   1     -14.709   5.649 -14.344  1.00 62.14           C  \nATOM      9  HB2 MET A   1     -14.371   4.944 -13.584  1.00 62.14           H  \nATOM     10  HB3 MET A   1     -14.487   5.235 -15.327  1.00 62.14           H  \nATOM     11  O   MET A   1     -12.035   6.072 -15.202  1.00 62.14           O  \nATOM     12  CG  MET A   1     -16.228   5.835 -14.230  1.00 62.14           

### Predict structure - use GPU-accelerated mmseqs2 for MSA
See [af2_nim_mmseqs2.log](https://github.com/hw-ju/Tutorial_AF2_NIM_Brev/blob/main/af2_nim_mmseqs2.log) for a sample of the complete log generated when running the code cell below as it is.
You should receive results in less than 40 minutes if you're using the compute configuration described in the tutorial.

In [7]:
import requests
import json

url = f'{AF2_HOST}/protein-structure/alphafold2/predict-structure-from-sequence'

# Replace with the actual sequence value
sequence = "MNVIDIAIAMAI" 

headers = {
    "content-type": "application/json"
}

data = {
    "sequence": sequence,
    "databases": ["uniref90", "small_bfd", "mgnify"],
    "algorithm": "mmseqs2"
}
# check wall time
%time response = requests.post(url, headers=headers, data=json.dumps(data))

# Check if the request was successful
if response.ok:
    print("Request succeeded:", response.json())
else:
    print("Request failed:", response.status_code, response.text)

CPU times: user 112 ms, sys: 26.3 ms, total: 138 ms
Wall time: 23min 55s
Request succeeded: ['ATOM      1  N   MET A   1     -14.401   7.955 -15.131  1.00 62.16           N  \nATOM      2  H   MET A   1     -13.906   8.817 -14.955  1.00 62.16           H  \nATOM      3  H2  MET A   1     -14.176   7.623 -16.058  1.00 62.16           H  \nATOM      4  H3  MET A   1     -15.395   8.122 -15.061  1.00 62.16           H  \nATOM      5  CA  MET A   1     -13.992   6.942 -14.142  1.00 62.16           C  \nATOM      6  HA  MET A   1     -14.177   7.295 -13.128  1.00 62.16           H  \nATOM      7  C   MET A   1     -12.502   6.776 -14.316  1.00 62.16           C  \nATOM      8  CB  MET A   1     -14.743   5.622 -14.361  1.00 62.16           C  \nATOM      9  HB2 MET A   1     -14.533   5.239 -15.360  1.00 62.16           H  \nATOM     10  HB3 MET A   1     -14.395   4.894 -13.628  1.00 62.16           H  \nATOM     11  O   MET A   1     -12.067   6.035 -15.190  1.00 62.16           O  \nATOM

In [8]:
result = response.json()

In [9]:
# The current version of AF2 NIM will run five models for structure prediction
len(result)

5

In [10]:
result[0][:400]

'ATOM      1  N   MET A   1     -14.401   7.955 -15.131  1.00 62.16           N  \nATOM      2  H   MET A   1     -13.906   8.817 -14.955  1.00 62.16           H  \nATOM      3  H2  MET A   1     -14.176   7.623 -16.058  1.00 62.16           H  \nATOM      4  H3  MET A   1     -15.395   8.122 -15.061  1.00 62.16           H  \nATOM      5  CA  MET A   1     -13.992   6.942 -14.142  1.00 62.16          '

### Save a predicted structure in a PDB file

In [19]:
import os
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

In [20]:
!ls .

af2_nim.log	new_directory  ngccli_linux.zip  output
call_API.ipynb	ngc	       nvidia-workbench  x_pred_structures


In [21]:
fp = os.path.join(output_dir, "predicted_protein.pdb")
with open(fp, "w") as f:
    f.write(result[0])

### Visualize the predicted structure

In [22]:
!pip install py3dmol

Defaulting to user installation because normal site-packages is not writeable


In [23]:
!pip show py3dmol

Name: py3Dmol
Version: 2.4.2
Summary: An IPython interface for embedding 3Dmol.js views in Jupyter notebooks
Home-page: https://3dmol.org
Author: David Koes
Author-email: dkoes@pitt.edu
License: MIT
Location: /home/ubuntu/.local/lib/python3.10/site-packages
Requires: 
Required-by: 


In [24]:
import sys
sys.path.append("/home/ubuntu/.local/lib/python3.10/site-packages")

import py3Dmol

In [25]:
def load_protein(pdb_file_path, width=800, height=600):

    """
    Load a protein structure from a PDB file and display it using py3Dmol.
    pdb_file_path: str, path to the PDB file
    width: int, width of the viewer in pixels
    height: int, height of the viewer in pixels
    return: py3Dmol.view object
    """

    with open(pdb_file_path) as ifile:
        pdb_data = "".join([x for x in ifile])

    view = py3Dmol.view(width=width, height=height)
    view.addModelsAsFrames(pdb_data)

    for line in pdb_data.split("\n"):
        split = line.split()
        if len(split) == 0 or split[0] != "ATOM":
            continue
        # Assuming the B-factor is at position 10 (you may need to adjust this based on your PDB format)
        b_factor = float(split[10])
        if b_factor > 90:
            color = "blue"
        elif 70 <= b_factor <= 90:
            color = "cyan"
        elif 50 <= b_factor < 70:
            color = "yellow"
        else:
            color = "orange"

        # Atom serial numbers typically start from 1, hence idx should be used directly
        idx = int(split[1])

        # Style should be set per atom id
        view.setStyle({'model': -1, 'serial': idx}, {"cartoon": {'color': color}})
    view.zoomTo()
    return view

In [26]:
view = load_protein(
    pdb_file_path = fp,
    width=800,
    height=500
)
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.